In [ ]:
import json
import requests
import html5lib
import bs4
from bs4 import BeautifulSoup

In [ ]:
# start the llm
#> ollama run llama3

In [ ]:
def get_llama_response(query, corpus_of_documents):
    # create the prompt
    prompt = """
    You are a bot that makes recommendations for holidays. You answer with facts, highlighting pros and cons.
    These are the recommended holidays: {corpus_of_documents}
    These are the tour names: {tours}
    The user input is: {user_input}
    Compile a recommendation to the user based on the recommended activities and the user input.
    """
    # setup llama3
    url = 'http://localhost:11434/api/generate'
    data = {
        "model": "llama3",
        "prompt": prompt.format(user_input=user_input, corpus_of_documents=corpus_of_documents, tours=tours)
    }
    headers = {'Content-Type': 'application/json'}
    response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)

    # generate and return the response
    full_response = []
    try:
        count = 0
        for line in response.iter_lines():
            if line:
                decoded_line = json.loads(line.decode('utf-8'))
                
                full_response.append(decoded_line['response'])
    finally:
        response.close()
    
    return full_response

In [ ]:
tour_urls = ['https://www.explore.co.uk/holidays/machu-picchu-trek',
             'https://www.explore.co.uk/holidays/vietnam-historic-tour',
             'https://www.explore.co.uk/holidays/cycling-holiday-morocco']

In [ ]:
contents = []
tours = []
for index, url in enumerate(tour_urls):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html5lib')

    tours.append(soup.title.get_text().replace('\n', ' ').replace('\t', ''))
    table = soup.find('div', attrs = {'id':'itinerary'})

    itinerary = []  # a list to store quotes
    for row in table.findAll('div', attrs = {'class':'pr-i-desc'}): 
        itinerary.append(row.p.get_text())

    contents.append(itinerary)

In [ ]:
corpus_of_documents = []
for item in contents:
    full_details = ' '.join(item)
    corpus_of_documents.append(full_details)

In [ ]:
user_input = "I'd like to visit South America and visit Machu Picchu"

In [ ]:
test_output = get_llama_response(user_input, corpus_of_documents)

In [ ]:
print(''.join(test_output))